# AirBnB in Athens : How does it influence our city?

---

> Dimitrios Kokkotas, 8200076 <br />
> Dept of Management Science and Technology <br />
> Athens University of Economics and Business <br />
> dtmkokkotas@gmail.com

* The current notebook performs an in depth analysis on AirBnB Athens data
* Tha data are collected by [Inside AirBnB](http://insideairbnb.com)
* You may visit Inside AirBnB's **data dictionary**, that provides an overview of the data structure and characterisitics
* Assumptions regarding the data used, can be found here: [Data Assumptions](http://insideairbnb.com/data-assumptions)
* Our core libraries : [pandas](https://pandas.pydata.org/) for our analysis and [matplotlib](https://matplotlib.org/) for visualizations

## Importing the needed packages
<br>
At first, we import several libraries, with them serving analysis, visualization and statistics models' estimation needs

In [ ]:
import folium
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import spearmanr
from folium.plugins import MarkerCluster

<br>
As we are working with a large number of columns within the dataframes, we may need to fully inspect them without truncation.
<br><br>

In [ ]:
pd.set_option('display.max_columns', None)

## Reading our data
<br>
Our next step is to obtain several useful CSV files, fetched from our [source](http://insideairbnb.com/get-the-data), and place them under the `data` folder. What are the data about?

* `listings.csv` : embodies summary information and metrics for listings in Athens 
* `reviews.csv`  : holds summary review data and listing ID
* `calendar.csv`      : provides detailed calendar data 
* `neighbourhoods.csv`: stores a neighbourhood list for geo filter

It's worth mentioning that there are also some corresponding to the listings (`listings-detailed.csv`) and  reviews (`reviews-detailed.csv`) files that hold more detailed information. At the moment, we just use the summary ones for the analysis needs.


We load the CSV files and assign them into DataFrame objects afterwards.
<br><br>

In [ ]:
listings = pd.read_csv('data/listings.csv')

neighbourhoods = pd.read_csv('data/neighbourhoods.csv')

# load 'date' column as date and time (by default is set as object type)
reviews = pd.read_csv('data/reviews.csv',
                     parse_dates=['date'])

# load 'date' column as date and time (by default is set as object type)
calendar = pd.read_csv('data/calendar.csv',
                      parse_dates=['date'])

<br>
We can now validate that the type of the `calendar.date` and `review.date` is `datetime`.
<br><br>

In [ ]:
calendar.dtypes['date']

In [ ]:
reviews.dtypes['date']

<br>
Gaining familiarity with the dataframes, we may overview our data via useful methods provided by pandas.
<br><br>

In [ ]:
listings.head(2)

In [ ]:
calendar.tail(2)

In [ ]:
neighbourhoods.shape

In [ ]:
calendar.info()

<br>And many many more.....



## Examining the popularity of Athens' neighbourhoods


To answer the question, we need to define those metrics, that properly estimate and assess the popularity of each neighbourhood. Inspecting the data filled in the `listing` dataframe, we collect two metrics, that seem representative for that purpose:

* number of listings per neighbourhood
* number of reviews  per neighbourhood

But first things first. Before calculating the above metrics, we shall group pur data by the neighbourhoods.<br><br>
Essentially, a `groupby` operation involves the combination of *splitting the object*, *applying a function* and finally *combining the results*.<br>

In [ ]:
neighbourhood_grp = listings.groupby(['neighbourhood'])

<br>
Now, for each and every neighbourhood, we can show the number of listings and reviews, sorted in a descending order. The higher the neighbourhood ranks, the more popular it is.
<br><br>

In [ ]:
# both of the following return a Series data structure

num_of_listings = neighbourhood_grp.size().sort_values(ascending=False) # count number of elements in each group

num_of_reviews = neighbourhood_grp['number_of_reviews'].sum().sort_values(ascending=False)

In [ ]:
num_of_listings

In [ ]:
num_of_reviews

<br>
Based on the above results, we understand that the two metrics depict rankings with **strong similarity**, but also show some **minor differencies**<br>
(see `ΑΓΙΟΣ ΚΩΝΣΤΑΝΤΙΝΟΣ-ΠΛΑΤΕΙΑ ΒΑΘΗΣ` neighbourhood).

Extending our analysis and deelving into the similarity rate of the results, that each metric delivered, we make use of the *Spearman's Correlation Coefficient* method to determine the association between them.
<br>

In [ ]:
print('Spearman', spearmanr(num_of_listings, num_of_reviews)) # print correlation coefficient and p_value

So, evaluating the results:

* **statistic is very close to 1** : there is a very strong positive monotonic relationship between the two series

* **p-value is very close to 0** : the correlation is highly statistically significant and the association is unlikely to be due to random chance.

All in all, both:
* results' observation and
* statistics' estimation,<br>

tend to demonstrate a high score of similarity between the two rankings

## Time for some descriptives

We are asked to create a table with the prices per neighbourhood. The prices information should contain:

* median price per neighbourhood
* mean price per neighbourhood
* standard deviation per neighbourhood
* number of listings per neighbourhood

Firstly, we make use of the existing `neighbourhood_grp` (*GroupBy* object), which holds a collection of data split into Athen's neighbourhood groups. 
<br><br>
How are the requested descriptives calculated?
<br><br>
By performing some aggregation functions on each group, and assign the results to a table.

In [ ]:
descriptives = neighbourhood_grp['price'].agg(['median', 'mean', 'std', 'size'])

As requested, we sort the contents of the table in descending order by the `median` statistical measure.

In [ ]:
descriptives = descriptives.sort_values(by='median', ascending=False)

Following up on the sorting operation, we make some final adjustments.

In [ ]:
descriptives = descriptives.reset_index()

In [ ]:
descriptives

## Visualizing the availability of listings per day

We are asked to create a plot with the number of listings available per day, on the $y$ axis, and the date, on the $x$ axis.

We are working on the `calendar` dataframe, which holds the data in reference to the listings, dates, and availabilities. 
<br><br>
Let's show some indicative rows, in order to get familiar with the structure. 
<br>

In [ ]:
calendar.head(2)

<br>
Since we are looking for availability, we need to filter out each and every occurrence of listings with no availability (`available ='f'`). 
<br><br>
That being the case, we assign the filtered dataframe to a new one called  `calendar_is_available`.
<br><br>

In [ ]:
calendar_is_available = calendar[(calendar['available'] == 't')]

<br>
Later, we group the data of the updated dataframe by date, so we can eventually count the number of listings available for each day.
<br><br>

In [ ]:
date_grp = calendar_is_available.groupby(['date'])

In [ ]:
available_per_day = date_grp['listing_id'].count()

In [ ]:
available_per_day # the values on the y-axis of the plot

<br>
Using the existing index of the `available_per_day` Series, we assign the unique identifiers to a new Series collection, called `dates`.
<br><br>

In [ ]:
dates = available_per_day.index

In [ ]:
dates # the values on the x-axis of the plot

<br><br>
Now we are ready to visualize our data.
<br><br>

In [ ]:
plt.style.use('seaborn-v0_8')

plt.plot_date(dates, available_per_day, linestyle='solid') # draw a line plot of data points with dates

plt.gcf().autofmt_xdate() # make sure that the dates in x-axis are not overcrowded and ensure readability

plt.title('Availability of Listings Across Time')
plt.xlabel('Date')
plt.ylabel('Number of Available Listings ')

plt.tight_layout()

plt.show()

## Reviews, Occupacy per Listing, Average Income

We are asked to create a histogram, visualizing the number of reviews per listing.<br><br>We will collect data from the `listings` dataframe, as both the `id` of the listing and it's `number of reviews`, are recorded there.<br>

We can validate that within the `listings` dataframes, the column `id` contains unique values.

In [ ]:
listings.id.nunique() == len(listings.id)

<br>
Given the above statement, there is no need for grouping the data by the listing id, and we can find the number of reviews per listing:
<br><br>

In [ ]:
reviews_per_listing = listings['number_of_reviews'] # returns a Series containing the num of reviews for each listing

In [ ]:
reviews_per_listing # each value will be 'binned'

<br>
As declared, the histogram should bin the number of reviews, from the smallest amount of reviews in a listing, to the maximum number of reviews in a listing rounded up to the closest hundred.
<br><br>
So we need to calculate some metrics, in order to determine the bins' characteristics:

* We shall find the `min`number of reviews 

In [ ]:
min_reviews = listings['number_of_reviews'].min()

* We shall find the `max` number of reviews rounded up to the closest hundred

In [ ]:
max_reviews = listings['number_of_reviews'].max()

In [ ]:
max_reviews

<br>
We are going to use the `ceil()` method provided by NumPy to round the `max_reviews` up to the closest hundred.
<br><br>

In [ ]:
max_reviews_rounded = int(np.ceil(max_reviews / 100) * 100)

In [ ]:
max_reviews_rounded

<br>
By completing the aforementioned steps, we can proceed to the specification of the bins:
<br><br>

In [ ]:
bins = np.arange(0, max_reviews_rounded + 100, 100)

In [ ]:
plt.hist(reviews_per_listing, bins=bins, edgecolor='black',color='blue')

plt.xlabel('Number of Reviews')
plt.ylabel('Number of Listings')
plt.title('Number of Reviews per Listing')

plt.xticks(np.arange(0, max_reviews_rounded + 100, 100))
plt.yticks(np.arange(0, 12000, 1000))

plt.tight_layout()

plt.show()

<br>
Moving forward, we need to calculate the average occupacy per listing per month.<br>
We will be working on the `reviews` dataframe, which contains a large size of data in reference to the listings and  the reviews' dates.

Our timeframe for the calculations of the averages, will be the entire year of 2022, since it's the most recent year containing full information. <br>Thus, we are filtering out outdated rows.
<br><br>

In [ ]:
reviews_rcnt = reviews[(reviews['date'].dt.year == 2022)]

How the data will look like?

In [ ]:
reviews_rcnt.head(2)

<br>
At first, we group our data by the `listing_id`, to proceed with the needed calculations.
<br><br>

In [ ]:
listing_grp = reviews_rcnt.groupby(['listing_id'])

Note : At this point, we assume that half of the bookings result in reviews.<br>
In order to calculate the average occupancy per listing per month, we need to compute the following metrics.
<br>

* the average reviews per listing per month

In [ ]:
avg_rev_per_month = listing_grp.size() / 12
avg_rev_per_month

* the average bookings per listing per month

In [ ]:
avg_bookings_per_month = avg_rev_per_month * 2
avg_bookings_per_month

* the average occupancy per listing per month<br>
<br>This metric is calculated through: the number of average booked days divided by the number of bookable days.


In [ ]:
avg_booked_days = avg_bookings_per_month * 3 # since each booking is a three night stay

In [ ]:
avg_occupancy_per_month = avg_booked_days / 30
avg_occupancy_per_month

<br><br>We are asked to find the average income per month, using the average price charged throughout all listings.
<br><br>Since we are talking about income, we shall group our data by the hosts, in order to find the total number of<br> listings for each host as well the average price for his/her listings.
<br>

Let's break down the problem: <br>In order to calculate the average income per host per month, we need to multiply the number of average booked days<br>(which we have already calculated as `avg_booked_days`) with the average prices for the host's listings.<br><br>
At first, we group our data by the host.

In [ ]:
host_grp = listings.groupby(['host_id'])

Then, we perform an aggregation operation upon each hosts' listings' prices.

In [ ]:
avg_price_per_host = host_grp['price'].mean()

At this point, we need for every host to find the average booked days per month for each and every listing owned. The aforementioned information is stored within the `avg_booked_days` Series. <br>

In [ ]:
avg_booked_days

So, we procced to a merging operation, in order to bring the needed data alongside with the `host_id`.
<br>
Be patient with the long and tedious naming here, but we try to be as descriptive ass possible.

In [ ]:
hosts_with_listings = host_grp['id'].value_counts().reset_index()

In [ ]:
hosts_with_listings

In [ ]:
listings_with_avg_booked_days = avg_booked_days.reset_index()

In [ ]:
listings_with_avg_booked_days.rename(columns={'listing_id' : 'id', 0 : 'avg_booked_days'}, inplace=True)

In [ ]:
listings_with_avg_booked_days

In [ ]:
hosts_with_avg_booked_days = pd.merge(hosts_with_listings, listings_with_avg_booked_days, on='id')

In [ ]:
hosts_with_avg_booked_days

In [ ]:
hosts_with_avg_booked_days.groupby(['host_id'])['avg_booked_days'].sum()

<br>
It's now time for the multiplication.
<br>

In [ ]:
hosts_with_avg_booked_days_and_avg_prices = pd.merge(hosts_with_avg_booked_days, avg_price_per_host, on='host_id')

In [ ]:
hosts_with_avg_booked_days_and_avg_prices

In [ ]:
result = hosts_with_avg_booked_days_and_avg_prices.groupby('host_id').apply(lambda x: (x['avg_booked_days'] * x['price']).sum()).reset_index(name='result')

In [ ]:
result # the average income per host per month

## Listings per Room Type

Moving on, we wish to show the number of listings per room type.
<br>
That being the case, we need at first to group our data in `listings` dataframe by the `room_type`.

In [ ]:
room_grp = listings.groupby(['room_type'])

<br>
We present the number of listings per room type in both:

* absolute numbers and

In [ ]:
room_grp['id'].count().abs()

* percentages

In [ ]:
(room_grp['id'].count() / len(listings.id)) * 100

<br>
Furthermore, we need to show the number of listings per neighrbourhood and room type.
<br><br>

In [ ]:
neighbour_room_grp = listings.groupby(['neighbourhood', 'room_type'])

In [ ]:
neighbour_room_grp['id'].count()

## Prices per Day



We are asked to create a table with the average price per listing per day.<br>
While working on the `calendar`dataframe we observe that the prices are not registered in a numeric type, but the type of them is object.<br> So, we proceed to the necessary data casting operations.

In [ ]:
calendar['price'] = calendar['price'].str.replace('$', '').str.replace(',', '').astype(float)

<br>
Moving on, we group our data by the id of the listings, and then assign the average prices to a table
<br><br>

In [ ]:
listing_groups = calendar.groupby(['listing_id'])

In [ ]:
avg_prices_per_day = listing_groups['price'].mean() # since each listing stores values for 365 days

In [ ]:
avg_prices_per_day

<br><br>
It's time for visualization. We demonstrate  the distribution of the listings into the different average (per day) price ranges, via a histogram diagram.
<br><br>

In [ ]:
plt.hist(avg_prices_per_day, bins=[0, 50, 100, 150, 200, 250], edgecolor='black', color='skyblue')
plt.title('Price Distribution')
plt.xlabel('Price Range')
plt.ylabel('Number of Listings')
plt.xticks([25, 75, 125, 175, 225])
plt.grid(axis='y', linestyle='--', alpha=0.7)

## Listings per Host

Our next task is to create a table with the number of listings per host, in descending order and then visualize it.

We are making use of the existing DataFrame GroupBy object `host_grp` anc count the number of listings for each host

In [ ]:
listings_per_host = host_grp.size().sort_values(ascending=False)

We make some further adjustments to the table.

In [ ]:
listings_per_host = listings_per_host.reset_index()

In [ ]:
listings_per_host.rename(columns={0 : 'counts'}, inplace=True)

In [ ]:
listings_per_host.head(5)

<br>
We are now ready to visualize the the `listings_per_host` table in:

* linear scale

In [ ]:
plt.plot(listings_per_host.index + 1, listings_per_host['counts'], marker='o', linestyle='-', markersize=8)

plt.title("Host Ranking on a Number of Listings Basis")
plt.xlabel("Rank")
plt.ylabel("Number of Listings")

plt.grid(True)
plt.tight_layout()

In [ ]:
plt.plot(listings_per_host.index + 1, listings_per_host['counts'], marker='o', linestyle='-', markersize=8)

plt.title("Host Ranking on a Number of Listings Basis")
plt.xlabel("Host")
plt.ylabel("Number of Listings")

plt.grid(True)
plt.tight_layout()

plt.xscale('log')

## Visualize Athen's listings with Folium

Our final task is to draw an interactive map, so we can show the listings ovelayed on Athens and for that purpose we are going to use Folium library.<br>

At first we instantiate a map object for a given location, that being Athen's coordinates

In [ ]:
athens_coordinates = [37.9838, 23.7275]

m = folium.Map(location=athens_coordinates, zoom_start=12, max_zoom=15)

<br><br>
Proceeding, we instantiate one single marker cluster to map.Using marker clusters, provides us a convenient way of demonstrating markers<br> in groups and helps us avoid overwhelming map info display. <br><br>Essentially, marker clusters are designed to group multiple map markers that are close to each other into a single cluster marker. <br><br>Furthermore, they indicate how many markers are present in that location.
<br><br>

In [ ]:
marker_cluster = MarkerCluster().add_to(m)

As we need to shape our map with multiple information (such as the listing's name, or the pricing data), we iterate through the values of `listings` dataframe, that provide the desired data.

* we assign to a `description` variable a formatted string, that collects info from the dataframe and will customize the description of the marker's pop-up

* eventually, we set the marker's arguments properly, so after the loop completes, each and every marker will have been succesfully registered and displayed in tha map

In [ ]:
for index, row in listings.iterrows():
    description = f"<strong>{row['name']}</strong><br>Host: {row['host_name']}<br>Price: ${row['price']} per night"
    folium.Marker([row['latitude'], row['longitude']], tooltip=row['name'], popup=description).add_to(marker_cluster)

As plenty resources will be committed for the map to open smoothly, we save it in an external HTML file.

In [ ]:
m.save('airbnb_athens.html')